In [24]:
import pandas as pd
import numpy as np
data = pd.read_csv("../data/salary_data.csv")


In [25]:
print(data.describe())
print(data.info())

        Unnamed: 0          Age  Years of Experience         Salary
count  6704.000000  6702.000000          6701.000000    6699.000000
mean   3351.500000    33.620859             8.094687  115326.964771
std    1935.422435     7.614633             6.059003   52786.183911
min       0.000000    21.000000             0.000000     350.000000
25%    1675.750000    28.000000             3.000000   70000.000000
50%    3351.500000    32.000000             7.000000  115000.000000
75%    5027.250000    38.000000            12.000000  160000.000000
max    6703.000000    62.000000            34.000000  250000.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6704 entries, 0 to 6703
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           6704 non-null   int64  
 1   Age                  6702 non-null   float64
 2   Gender               6702 non-null   object 
 3   Education Level      6701 n

In [26]:
# Drop the unnecessary columns
cols= [0, 8]
data = data.drop(data.columns[cols], axis=1)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6704 entries, 0 to 6703
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  6702 non-null   float64
 1   Gender               6702 non-null   object 
 2   Education Level      6701 non-null   object 
 3   Job Title            6702 non-null   object 
 4   Years of Experience  6701 non-null   float64
 5   Salary               6699 non-null   float64
 6   Country              6704 non-null   object 
dtypes: float64(3), object(4)
memory usage: 366.8+ KB
None


In [27]:
data.isnull().sum()


Age                    2
Gender                 2
Education Level        3
Job Title              2
Years of Experience    3
Salary                 5
Country                0
dtype: int64

In [28]:
# Drop missing values
data.dropna(inplace=True)

In [29]:
print("Columns in data:", data.columns.tolist())

duplicate_count = data.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

# Drop duplicate rows
data.drop_duplicates(inplace=True)


Columns in data: ['Age', 'Gender', 'Education Level', 'Job Title', 'Years of Experience', 'Salary', 'Country']
Number of duplicate rows: 3104


In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

numerical_columns = ['Age', 'Years of Experience', 'Salary']  
categorical_columns = ['Gender', 'Country', 'Education Level'] 

# Create the ColumnTransformer
preprocessor = ColumnTransformer([
    ('num', MinMaxScaler(), numerical_columns),  # MinMax scaling for numerical columns
    ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_columns)  # One-hot encoding for categorical columns
])

# Apply transformation
transformed_data = preprocessor.fit_transform(data)

column_names = (
    numerical_columns +  
    list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns)) 
)
processed_data = pd.DataFrame(transformed_data, columns=column_names)

print(processed_data.head())


        Age  Years of Experience    Salary  Gender_Male  Gender_Other  \
0  0.268293             0.147059  0.359103          1.0           0.0   
1  0.170732             0.088235  0.258963          0.0           0.0   
2  0.585366             0.441176  0.599439          1.0           0.0   
3  0.365854             0.205882  0.238935          0.0           0.0   
4  0.756098             0.588235  0.799720          1.0           0.0   

   Country_Canada  Country_China  Country_UK  Country_USA  \
0             0.0            0.0         1.0          0.0   
1             0.0            0.0         0.0          1.0   
2             1.0            0.0         0.0          0.0   
3             0.0            0.0         0.0          1.0   
4             0.0            0.0         0.0          1.0   

   Education Level_Bachelor's Degree  Education Level_High School  \
0                                0.0                          0.0   
1                                0.0                    

In [31]:
from scipy.stats import zscore

def remove_outliers(df, columns):
    cleaned_df = df.copy()
    outlier_info = {}

    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        iqr_mask = (df[col] >= lower_bound) & (df[col] <= upper_bound)

        z_scores = np.abs(zscore(df[col]))
        z_mask = z_scores <= 3  

        combined_mask = iqr_mask & z_mask  
        outlier_info[col] = {'IQR Outliers': (~iqr_mask).sum(), 'Z-Score Outliers': (~z_mask).sum(), 'Total Outliers': (~combined_mask).sum()}


        cleaned_df = cleaned_df.loc[iqr_mask & z_mask]  
        

    return cleaned_df


numerical_columns = ['Age', 'Years of Experience', 'Salary'] 


original_size = processed_data.shape[0]

cleaned_data = remove_outliers(processed_data, numerical_columns)

new_size = cleaned_data.shape[0]
outliers_removed = original_size - new_size

print(f"Total outliers removed: {outliers_removed}")
print(f"Original dataset size: {original_size}")
print(f"New dataset size: {new_size}")


Total outliers removed: 50
Original dataset size: 3594
New dataset size: 3544


In [32]:
cleaned_data

,Age,Years of Experience,Salary,Gender_Male,Gender_Other,Country_Canada,Country_China,Country_UK,Country_USA,Education Level_Bachelor's Degree,Education Level_High School,Education Level_Master's,Education Level_Master's Degree,Education Level_PhD,Education Level_phD
0,0.268293,0.147059,0.359103,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.170732,0.088235,0.258963,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.585366,0.441176,0.599439,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.365854,0.205882,0.238935,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.756098,0.588235,0.799720,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589,0.292683,0.205882,0.359103,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3590,0.390244,0.176471,0.299019,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3591,0.682927,0.588235,0.799720,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3592,0.268293,0.088235,0.198878,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [33]:
cleaned_data.to_csv('../data/joint_data_collection.csv', index=False)
print("Cleaned data saved to 'joint_data_collection.csv'")


Cleaned data saved to 'joint_data_collection.csv'


In [34]:
from sklearn.model_selection import train_test_split

# Split the data into training (80%) and testing (20%) sets
train_data, test_data = train_test_split(processed_data, test_size=0.2, random_state=42)

train_data.to_csv('../data/training_data.csv', index=False)
test_data.to_csv('../data/test_data.csv', index=False)

print(f"Training data saved to 'training_data.csv' with {train_data.shape[0]} rows.")
print(f"Test data saved to 'test_data.csv' with {test_data.shape[0]} rows.")

Training data saved to 'training_data.csv' with 2875 rows.
Test data saved to 'test_data.csv' with 719 rows.


In [35]:
# Randomly select one entry from the test dataset and store it to activation_data.csv
activation_data = test_data.sample(n=1, random_state=42)

activation_data.to_csv('../data/activation_data.csv', index=False)

print("One test entry saved to 'activation_data.csv'")


One test entry saved to 'activation_data.csv'


In [36]:
test_data

,Age,Years of Experience,Salary,Gender_Male,Gender_Other,Country_Canada,Country_China,Country_UK,Country_USA,Education Level_Bachelor's Degree,Education Level_High School,Education Level_Master's,Education Level_Master's Degree,Education Level_PhD,Education Level_phD
2834,0.414634,0.411765,0.679551,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
315,0.634146,0.588235,0.719607,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3538,0.170732,0.117647,0.218906,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3541,0.268293,0.088235,0.158822,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
439,0.195122,0.176471,0.719607,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,0.585366,0.411765,0.559383,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3322,0.341463,0.205882,0.359103,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2378,0.195122,0.088235,0.194873,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2722,0.439024,0.382353,0.579411,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [37]:
train_data


,Age,Years of Experience,Salary,Gender_Male,Gender_Other,Country_Canada,Country_China,Country_UK,Country_USA,Education Level_Bachelor's Degree,Education Level_High School,Education Level_Master's,Education Level_Master's Degree,Education Level_PhD,Education Level_phD
1123,0.707317,0.558824,0.799720,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
177,0.414634,0.294118,0.319047,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1475,0.121951,0.088235,0.289601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1108,0.317073,0.205882,0.519327,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1407,0.219512,0.235294,0.430783,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,0.317073,0.235294,0.559383,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1294,0.097561,0.058824,0.360100,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
860,0.146341,0.058824,0.278991,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3507,0.073171,0.029412,0.138794,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
